## Pipeline for House Price Prediction

### Problem Statement
Acquiring properties is common in our society today. However, a guildline for interested/prospected buyers to help them get a good value for their money seems to be lacking and buyers are left at their fate to gamble different options with their hard earned money. This project seeks to provide a model to guide buyers predict the price of a house based on their choices' features of a house.

### Data:
The data was collected within the 3 months, the feautures include city, number of bedrooms, number of bathrooms, square of living area, square of basement, number of floors, waterfront, number of views, year built, year renovated, etc

### Installing the necessary libraries

In [2]:
!python -m pip install --user --upgrade pip

!pip3 install pandas==0.23.4 matplotlib==3.0.3 scipy==1.2.1 scikit-learn==0.22 tensorflow==2.0 keras==1.2.2 --user

Requirement already up-to-date: pip in c:\users\miloh\appdata\roaming\python\python37\site-packages (20.2.4)


Restart the kernel before you proceed

In [1]:
import numpy as np
import pandas as  pd
import os
import matplotlib.pyplot as plt

# Evaluation
from sklearn.metrics import mean_squared_error

## Install Kubeflow pipelines SDK

In [2]:
# You may need to restart your notebook kernel after updating the kfp sdk
!pip3 install kfp --upgrade --user

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for kfp: filename=kfp-1.1.1-py3-none-any.whl size=221433 sha256=767933b861b0ad7de8af044c6cb26094be2f3c1b8bee7730c7ed52d7aa11bfa3
  Stored in directory: c:\users\miloh\appdata\local\pip\cache\wheels\41\38\6c\29aaec4c314a72c29ef05a82f3c8d0de06d447145d56d24f61
  Created wheel for kfp-server-api: filename=kfp_server_api-1.0.4-py3-none-any.whl size=104117 sha256=5091780b84066dc86dd56f1e007248c066ae7b23bae3391a9ce09e4451986982
  Stored in directory: c:\users\miloh\appdata\local\pip\cache\wheels\1f\46\3e\4c4154156f4278f818e0dc18a82c29f174d2695ec13a396413
  Created wheel for strip-hints: filename=strip_hints-0.1.9-py2.py3-none-any.whl size=20999 sha256=9c526d734cb0448a7d3

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

google-api-core 1.23.0 requires six>=1.13.0, but you'll have six 1.12.0 which is incompatible.


Check if the install was successful:

In [1]:
!which dsl-compile

'which' is not recognized as an internal or external command,
operable program or batch file.


## Setup

In [4]:
EXPERIMENT_NAME = 'stage-f-14-house-pricing pipeline'        # Name of the experiment in the UI
BASE_IMAGE = "tensorflow/tensorflow:latest-gpu-py3"    # Base image used for components in the pipeline

## Build the components

In [2]:
# Import Kubeflow SDK
import kfp
from kfp import compiler
import kfp.dsl as dsl
import kfp.components as comp
import os
import subprocess
import json

In [3]:
# where the outputs are stored
out_dir = "/home/jovyan/House_Pricing_Prediction/data/out/"

## Create a pipeline Function
## Preprocessing Function

In [6]:
@dsl.python_component(
    name='preprocess',
    description='preprocessing function for House Pricing',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)


def preprocess(data_path):
    
    import numpy as np
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    from sklearn.model_selection import KFold
    from sklearn.model_selection import train_test_split  # splitting the data
    import pandas as pd
    # Get data
    data = pd.read_csv('https://raw.githubusercontent.com/ugoiloh/stage-f-14-house-pricing/ugoiloh/data/data.csv')
    
    # drop unneccessary column
    data.drop(columns=['date','country', 'statezip', 'street'], inplace=True)
    
    #Filtering for prices that are not zero.
    data = data.query('price != 0')
    
    #Filtering for houses not zero for number of bedrooms and bathrooms
    data = data.query('bedrooms != 0' or 'bathrooms != 0')
    
    # Converting the city variable to numerical values.
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    data['city'] = le.fit_transform(data['city'])
    
    #year convert function
    def yr_col (col1, col2):
        if col1 == 0:
            col1 = col2
        else:
            col1
        return col1
    
    #Change year renovated column with zero entry to the year built.
    data['yr_renovated'] = data.apply(lambda x: yr_col(x['yr_renovated'], x['yr_built']), axis =1)
    
    #Filtering the outliers
    data = data[(
                (data['price'] <= 2000000) & 
                (data['price'] > 150000) & 
                (data['bathrooms'] <= 4.5) & 
                (data['condition'] > 2) & 
                (data['sqft_living'] > 700) & 
                (data['sqft_living'] <= 5000) & 
                (data['sqft_lot'] <= 50000) & 
                (data['sqft_above'] <= 5000) &
                (data['sqft_basement'] <= 5000) &
                (data['bedrooms'] <= 6) 
                )]
    
    #Filtering for multicollinearity
    data = data.drop(columns=['sqft_living', 'sqft_above'])
    
    # We normalise our dataset to a common scale using the min max scaler
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
    
    # split the data into X and y
    X = data.drop(['price'], axis=1)  # predictor
    y = data['price'] # target
    
    # Split the data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
    #output file to path
    np.savez_compressed(f'{data_path}/preprocessed-data.npz', 
                       X_train=X_train,
                       X_test=X_test,
                       y_train=y_train,
                       y_test=y_test)
    print("Preprocessing Done")

C:\Users\miloh\AppData\Local\Continuum\anaconda31\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  after removing the cwd from sys.path.


## Training Function
## Training the data with the Catboost Regressor

In [7]:
@dsl.python_component(
    name='train',
    description='training function for House Pricing',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)

def train(data_path, model_file):
    
    # Install all the dependencies inside the function
    import numpy as np
    
    import pickle
    import sys, subprocess;
    #subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'catboost==0.24.2'])
    import pandas as pd
    # import libraries for training
    #!pip3 install catboost
    from catboost import CatBoostRegressor
    
    #load the preprocessed data
    preprocessed_data = np.load(f'{data_path}/preprocessed-data.npz')
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    # Instantiating the model 
    model = CatBoostRegressor(verbose=0, n_estimators=100)
    
    # Fit the model to the training data
    model.fit(X_train,y_train)
    
    #Save the model to the designated 
    with open(f'{data_path}/{model_file}', 'wb') as file:
        pickle.dump(model, file)
        
    print("Model Trained")

C:\Users\miloh\AppData\Local\Continuum\anaconda31\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  after removing the cwd from sys.path.


In [8]:
@dsl.python_component(
    name='predict',
    description='prediction function for House Pricing',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)

def predict(data_path, model_file):
    
    import pickle     # python object for (de)serialization
    import sys, subprocess;
    import numpy as np
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'catboost==0.24.2'])
    # Evaluation metrics
    from sklearn.metrics import mean_squared_error, r2_score 
      
    # Load the saved trained model
    with open(f'{data_path}/{model_file}', 'rb') as file:
        model = pickle.load(file)
    
    #load the preprocessed data
    preprocessed_data = np.load(f'{data_path}/preprocessed-data.npz')
    X_test = preprocessed_data['X_test']
    y_test = preprocessed_data['y_test']
    X_train = preprocessed_data['X_train']
    y_train = preprocessed_data['y_train']
    
    #Evaluate the model and print the results
    model_pred = model.predict(X_test)
    
    # print the RMSE
    #print('Model \nRMSE score = {}' .format(np.sqrt(mean_squared_error(y_test, model_pred))))
            
    # print the RMSE
    #print('Model \nRMSE score = {}' .format(r2_score(y_test, model_pred)))
              
    print('Model Rsquared score: %.2f' % r2_score(y_test, model_pred))
    print("Model RMSE RMSE_test score: %0.3f" % np.sqrt(mean_squared_error(y_test, model_pred)))
    print("Model RMSE_train score: %0.3f" % np.sqrt(mean_squared_error(y_train, model.predict(X_train))))
    
    with open(f'{data_path}/model_result.txt', 'w') as result:
        result.write(" Prediction: {},\nActual: {} ".format(model_pred, y_test))
              
    print('Prediction has been saved successfully!')

C:\Users\miloh\AppData\Local\Continuum\anaconda31\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  after removing the cwd from sys.path.


### Build the components

In [9]:
# Create preprocess, train and predict lightweight components.
preprocess_op = comp.func_to_container_op(preprocess, base_image=BASE_IMAGE)
train_op = comp.func_to_container_op(train , base_image=BASE_IMAGE)
predict_op = comp.func_to_container_op(predict , base_image=BASE_IMAGE)

## Build Kubeflow Pipeline

In [10]:
#Create a client to enable communication with the Pipelines API server.
client = kfp.Client()

Failed to load kube config.


In [11]:
# domain-specific language 
@dsl.pipeline(
    name='House Prediction',
    description='End-to-end training to predict the price of a house'
)

# Define parameters to be fed into pipeline
def house_prediction_container_pipeline(
    data_path: str,
    model_file: str
):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="volume_creation",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWO)
    
    # Create house price preprocessing component
    price_preprocessing_container = preprocess_op(data_path).add_pvolumes({data_path: vop.volume})
    
    # Create house price training component.
    price_training_container = train_op(data_path, model_file) \
                                    .add_pvolumes({data_path: price_preprocessing_container.pvolume})
    
    # Create price prediction component.
    price_predict_container = predict_op(data_path, model_file) \
                                    .add_pvolumes({data_path: price_training_container.pvolume})
    
    # Print the result of the prediction
    House_Price_result_container = dsl.ContainerOp(
        name="House Price prediction",
        image='library/bash:4.4.23',
        pvolumes={data_path: price_predict_container.pvolume},
        arguments=['head', f'{data_path}/model_result.txt']
    )

## Run the Pipeline
Kubeflow Pipelines lets you group pipeline runs by Experiments.

In [12]:
DATA_PATH = '/mnt'
MODEL_PATH='house_price_predictor.pkl'

In [14]:
pipeline_func = house_prediction_container_pipeline

In [15]:
experiment_name = 'House_Price_Prediction'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH,
             "model_file":MODEL_PATH}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func,'{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)

C:\Users\miloh\AppData\Roaming\Python\Python37\site-packages\kfp\dsl\_container_op.py:1028: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


MaxRetryError: HTTPConnectionPool(host='localhost', port=80): Max retries exceeded with url: /apis/v1beta1/experiments (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000027E9A03E908>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))